In [1]:
!pip install duckdb
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 18.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.5/210.5 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 11.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [157]:
import pandas as pd
import numpy as np
import json


## Read and Scale Data

In [159]:
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv('/content/drive/MyDrive/municipality_bus_utilization.csv')


usage_scaler = MinMaxScaler()
total_capacity_scaler = MinMaxScaler()

## Data Visualizations

In [16]:
# from dataprep.eda import create_report
# create_report(df)

In [17]:
# import sweetviz as sv
# analyze_report = sv.analyze(df)
# analyze_report.show_html('report.html', open_browser=False)

## Run fast SQL queries on data

In [160]:
import duckdb

def run_query(query, table, silent=False):
    query = query.format(table=table)
    print(query)
    
    x = duckdb.sql(query)
    if not silent: print(x)
    return x


In [161]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Extraxct the time and date infromation from input table

In [162]:
query = """
select
    timestamp,
    make_date(
        datepart(['year', 'month', 'day'], timestamp)['year'], 
        datepart(['year', 'month', 'day'], timestamp)['month'],
        datepart(['year', 'month', 'day'], timestamp)['day']) as date,
    make_time(
        datepart(['hour', 'minute'], timestamp)['hour'],
        datepart(['hour', 'minute'], timestamp)['minute'],
        0.0
    ) as time,
    municipality_id,
    usage,
    total_capacity
from {table}
order by
    municipality_id,
    timestamp
"""

In [163]:
table_1 = run_query(query, "df")


select
    timestamp,
    make_date(
        datepart(['year', 'month', 'day'], timestamp)['year'], 
        datepart(['year', 'month', 'day'], timestamp)['month'],
        datepart(['year', 'month', 'day'], timestamp)['day']) as date,
    make_time(
        datepart(['hour', 'minute'], timestamp)['hour'],
        datepart(['hour', 'minute'], timestamp)['minute'],
        0.0
    ) as time,
    municipality_id,
    usage,
    total_capacity
from df
order by
    municipality_id,
    timestamp

┌─────────────────────┬────────────┬──────────┬─────────────────┬───────┬────────────────┐
│      timestamp      │    date    │   time   │ municipality_id │ usage │ total_capacity │
│      timestamp      │    date    │   time   │      int64      │ int64 │     int64      │
├─────────────────────┼────────────┼──────────┼─────────────────┼───────┼────────────────┤
│ 2017-06-04 07:59:42 │ 2017-06-04 │ 07:59:00 │               0 │   204 │           2813 │
│ 2017-06-04 08:25:42 │ 2017-06-04 │ 08:25:00 

# Extract the time block infromation from hour column and get the max of the two same block

In [164]:
query="""
    select
        date,
        time,
        case
            when datepart('minute', time) > 30
            then
                case when
                    abs(datediff(
                        'minute',
                        make_time(datepart('hour', time)+1,0,0),
                        time
                    )) < 15
                then 
                    datepart('hour', time)+1
                else
                    datepart('hour', time)
                end
            else
                datepart('hour', time)
        end as block,    
        municipality_id,
        usage,
        total_capacity
    from {table}
"""

In [165]:
table_2 = run_query(query, 'table_1')


    select
        date,
        time,
        case
            when datepart('minute', time) > 30
            then
                case when
                    abs(datediff(
                        'minute',
                        make_time(datepart('hour', time)+1,0,0),
                        time
                    )) < 15
                then 
                    datepart('hour', time)+1
                else
                    datepart('hour', time)
                end
            else
                datepart('hour', time)
        end as block,    
        municipality_id,
        usage,
        total_capacity
    from table_1

┌────────────┬──────────┬───────┬─────────────────┬───────┬────────────────┐
│    date    │   time   │ block │ municipality_id │ usage │ total_capacity │
│    date    │   time   │ int64 │      int64      │ int64 │     int64      │
├────────────┼──────────┼───────┼─────────────────┼───────┼────────────────┤
│ 2017-06-04 │ 07:59:00 │     8 │            

In [166]:
query = """
    select
        date,
        --time,
        block,
        municipality_id,
        max(usage) as max_usage,
        total_capacity
    from {table}
    group by
        municipality_id,
        date,
        block,
        total_capacity
    order by
        municipality_id,
        date,
        block
        --time
"""

In [167]:
table_3 = run_query(query, 'table_2')


    select
        date,
        --time,
        block,
        municipality_id,
        max(usage) as max_usage,
        total_capacity
    from table_2
    group by
        municipality_id,
        date,
        block,
        total_capacity
    order by
        municipality_id,
        date,
        block
        --time

┌────────────┬───────┬─────────────────┬───────────┬────────────────┐
│    date    │ block │ municipality_id │ max_usage │ total_capacity │
│    date    │ int64 │      int64      │   int64   │     int64      │
├────────────┼───────┼─────────────────┼───────────┼────────────────┤
│ 2017-06-04 │     8 │               0 │       247 │           2813 │
│ 2017-06-04 │     9 │               0 │       429 │           2813 │
│ 2017-06-04 │    10 │               0 │       534 │           2813 │
│ 2017-06-04 │    11 │               0 │       597 │           2813 │
│ 2017-06-04 │    12 │               0 │       614 │           2813 │
│ 2017-06-04 │    13 │               0 │   

## Fill missing data with average of 1 week before and after from missing date block

In [170]:
from datetime import timedelta, date, datetime

df_block = table_3.df()

max_date = max(df_block['date'])
min_date = min(df_block['date'])
blocks = sorted(df_block['block'].unique())

for m_id in range(10):

    total_capacity = df_block.loc[df_block['municipality_id'] == 0, ['total_capacity']]['total_capacity'][0]
    for d in range((max_date - min_date).days+1):
        cur_date = min_date + timedelta(days=d)
        for b in blocks:
            row = df_block.loc[
                ((df_block['municipality_id']== m_id) &
                (df_block['date'] == cur_date) &
                (df_block['block'] == b))
            ]
            
            if row.shape[0] == 0:
                w1_before = cur_date + timedelta(days=7)
                w1_after = cur_date + timedelta(days=7)
                w1_before_data = df_block.loc[
                                    (
                                        (df_block['municipality_id'] == m_id) &                                        
                                        (df_block['date'] == w1_before) &
                                        (df_block['block'] == b)
                                    ), 
                                    ['max_usage']].values[0][0]

                w1_after_data = df_block.loc[
                                    (
                                        (df_block['municipality_id'] == m_id) &
                                        (df_block['date'] == w1_after) &
                                        (df_block['block'] == b)
                                    ), 
                                    ['max_usage']].values[0][0]

                row = {
                    'date': cur_date,
                    'municipality_id':m_id, 
                    'block': b,
                    'max_usage': int((w1_before_data+w1_after_data)/2),
                    'total_capacity': total_capacity
                }
                df_block = df_block.append(row, ignore_index=True)

df_block.sort_values(by=['date', 'municipality_id', 'block'], inplace=True)


## Create time series data and handle the out of window size samples

In [171]:
query= """
select
    t3.*,
    w1_f.max_usage as max_usage_1F,
    w2_f.max_usage as max_usage_2F,
    w1_p.max_usage as max_usage_1P,
    w2_p.max_usage as max_usage_2P
from
    {table} as t3
left join
    {table} as w1_f 
on
    t3.date = w1_f.date - INTERVAL 7 DAY and
    t3.municipality_id = w1_f.municipality_id and
    t3.block = w1_f.block
left join
    {table} as w2_f 
on
    t3.date = w2_f.date - INTERVAL 14 DAY and
    t3.municipality_id = w2_f.municipality_id and
    t3.block = w2_f.block 
left join
    {table} as w1_p
on
    t3.date = w1_p.date + INTERVAL 7 DAY and
    t3.municipality_id = w1_p.municipality_id and
    t3.block = w1_p.block
left join
    {table} as w2_p
on
    t3.date = w2_p.date + INTERVAL 14 DAY and
    t3.municipality_id = w2_p.municipality_id and
    t3.block = w2_p.block
order by 
    t3.date
"""

shifted = run_query(query, 'df_block')

query ="""
select
    s.date,
    s.block,
    s.municipality_id,
    s.max_usage,
    case
        when
            s.max_usage_1F is NULL
        then s.max_usage
        else s.max_usage_1F
    end as max_usage_1F,
    case
        when s.max_usage_2F is NULL
        then
        case
            when s.max_usage_1F is NULL
            then s.max_usage
            else s.max_usage_2F
        end
        else s.max_usage_2F
    end as max_usage_2F,
    case
     when s.max_usage_1P is NULL
     then 
        s.max_usage
    else
        s.max_usage_1P
    end as max_usage_1P,
    case 
        when s.max_usage_2P is NULL
        then
            case
                when s.max_usage_1P is NULL
                then s.max_usage
                else s.max_usage_1P
            end
        else s.max_usage_2P
    end as max_usage_2P,
    s.total_capacity
from {table} as s
"""

shifted = run_query(query, 'shifted').df()


select
    t3.*,
    w1_f.max_usage as max_usage_1F,
    w2_f.max_usage as max_usage_2F,
    w1_p.max_usage as max_usage_1P,
    w2_p.max_usage as max_usage_2P
from
    df_block as t3
left join
    df_block as w1_f 
on
    t3.date = w1_f.date - INTERVAL 7 DAY and
    t3.municipality_id = w1_f.municipality_id and
    t3.block = w1_f.block
left join
    df_block as w2_f 
on
    t3.date = w2_f.date - INTERVAL 14 DAY and
    t3.municipality_id = w2_f.municipality_id and
    t3.block = w2_f.block 
left join
    df_block as w1_p
on
    t3.date = w1_p.date + INTERVAL 7 DAY and
    t3.municipality_id = w1_p.municipality_id and
    t3.block = w1_p.block
left join
    df_block as w2_p
on
    t3.date = w2_p.date + INTERVAL 14 DAY and
    t3.municipality_id = w2_p.municipality_id and
    t3.block = w2_p.block
order by 
    t3.date

┌─────────────────────┬───────┬─────────────────┬───┬──────────────┬──────────────┬──────────────┬──────────────┐
│        date         │ block │ municipality_id │ … │

## Convert date to seperate features like year (which same for all rows so will be excluded), month, n'th day of week  

In [172]:
query = """
    select 
        s.date,
        datepart('month', s.date) as month,
        strftime(s.date, '%a') as day,
        block, 
        municipality_id, 
        total_capacity, 
        max_usage,
        max_usage_1F, 
        max_usage_2F, 
        max_usage_1P, 
        max_usage_2P
    from {table} as s
"""

## Convert day to weekday id
df = run_query(query, 'shifted').df()
day2id = {day:i for i,day in enumerate(df.day.unique())}
df['day'] = df.day.apply(lambda x: day2id[x])



    select 
        s.date,
        datepart('month', s.date) as month,
        strftime(s.date, '%a') as day,
        block, 
        municipality_id, 
        total_capacity, 
        max_usage,
        max_usage_1F, 
        max_usage_2F, 
        max_usage_1P, 
        max_usage_2P
    from shifted as s

┌─────────────────────┬───────┬─────────┬───────┬───┬──────────────┬──────────────┬──────────────┬──────────────┐
│        date         │ month │   day   │ block │ … │ max_usage_1F │ max_usage_2F │ max_usage_1P │ max_usage_2P │
│      timestamp      │ int64 │ varchar │ int64 │   │    int64     │    double    │    int64     │    int64     │
├─────────────────────┼───────┼─────────┼───────┼───┼──────────────┼──────────────┼──────────────┼──────────────┤
│ 2017-06-04 00:00:00 │     6 │ Sun     │     8 │ … │          295 │        304.0 │          247 │          247 │
│ 2017-06-04 00:00:00 │     6 │ Sun     │     9 │ … │          443 │        458.0 │          429 │          429 │
│ 201

## Train Test Split

In [173]:
from datetime import datetime

X_cols = ['date', 'month' ,'day' ,'block', 'municipality_id', 'total_capacity', 
                     'max_usage_1F', 'max_usage_2F', 'max_usage_1P', 'max_usage_2P']
y_cols = ['date', 'municipality_id', 'max_usage']

X = df.loc[:, X_cols]
y = df.loc[:, y_cols]

test_datetime_start = datetime(year=2017, month=8, day=5)
test_datetime_end = datetime(year=2017, month=8, day=19)

X_train = X.loc[X['date'] < test_datetime_start, [x for x in X_cols if x != ['date']]]
X_test = X.loc[X['date'] >= test_datetime_start, [x for x in X_cols if x != ['date']]]
y_train = y.loc[X['date'] < test_datetime_start, y_cols]
y_test = y.loc[X['date'] >= test_datetime_start, y_cols]

X_train['date'] = X_train['date'].view(int)
X_test['date'] = X_test['date'].view(int)

## Model Training

In [174]:
import optuna
import xgboost as xgb

def tune_and_cross_validate(X, y, n_trials=100, n_splits=5, random_state=42, parameters=None):

    def objective(trial):
        if parameters is None:
            depth = trial.suggest_int("max_depth", 3, 10)
            params = {
                'booster': 'gbtree',
                'silent': 1,
                'metric': 'reg_squarederror',
                'objective': 'reg:squarederror',
                'eta': trial.suggest_float("eta", 0.01, 0.03),
                'colsample_bytree': trial.suggest_float("colsample_bytree", 0.4, 1.0),
                'subsample': trial.suggest_float("subsample", 0.6, 1.0),
                'min_child_weight': trial.suggest_int("min_child_weight", 1, 10),
                'max_depth': depth,
                'n_estimators': trial.suggest_int("n_estimators", 100, 1000),
                'num_leaves': trial.suggest_int("num_leaves", 2, 2 ** depth),
                'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
                'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
                'random_state': random_state,
                'gpu_id' : 0,
                'tree_method' : 'gpu_hist'
            }
        else:
            params = parameters

        dtrain = xgb.DMatrix(X, y)

        gbm = xgb.cv(params, dtrain, num_boost_round=1000, nfold=n_splits)
        
        return gbm["test-rmse-mean"].iloc[-1]

    study = optuna.create_study(directions=["minimize"])
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    print("Number of finished trials: {}".format(len(study.trials)))
    print("Best trial:")
    trial = study.best_trials[0]

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    return trial.params

## Train One Model for all

In [175]:
params = tune_and_cross_validate(X_train, y_train.loc[:, ['max_usage']], n_trials=50)
with open('/content/drive/MyDrive/params.json', 'w+') as f: json.dump(params, f, indent=4)

[I 2023-02-27 11:58:41,956] A new study created in memory with name: no-name-08d4013c-b10a-4ef7-a95b-e1242c1e5cbd
/usr/local/lib/python3.8/dist-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2023-02-27 11:58:58,687] Trial 0 finished with value: 96.6494338 and parameters: {'max_depth': 3, 'eta': 0.02549744695341413, 'colsample_bytree': 0.8944857857540429, 'subsample': 0.9548809921190293, 'min_child_weight': 2, 'n_estimators': 108, 'num_leaves': 8, 'lambda': 3.7559491134074687e-06, 'alpha': 0.0013044748359196427}. Best is trial 0 with value: 96.6494338.
[I 2023-02-27 11:59:19,555] Trial 1 finished with value: 78.33122080000001 and parameters: {'max_depth': 5, 'eta': 0.028557244937604795, 'colsample_bytree': 0.5446094961043826, 'subsample': 0.6490659164127698, 'min_child_weight': 6, 'n_estimators': 944, 'num_leaves': 24, 'lambda': 1.9883852045409833e-05, 'alpha': 0.016658604935145083}. Best is trial 1 with value: 78.33122080000001.
[I 2023-02-27 11:59:48,448] Trial 2 finished with value: 74.7147126 and parameters: {'max_depth': 7, 'eta': 0.022115271818232352, 'colsample_bytree': 0.6063216042866038, 'subsample': 0.6642144910830979, 'min_child_weight': 7, 'n_estimators': 338

## Train model per municipality

In [190]:
params = {}
for id in range(10):
    params[id] = tune_and_cross_validate(X_train.loc[X_train['municipality_id']==id], 
                                         y_train.loc[y_train['municipality_id']==id, ['max_usage']],
                                         n_trials = 10)

with open('/content/drive/MyDrive/params_multi.json', 'w+') as f: json.dump(params, f, indent=4)



[I 2023-02-27 12:54:18,263] A new study created in memory with name: no-name-966aac41-8acc-4ef1-9c7b-bcfec65463a7
/usr/local/lib/python3.8/dist-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2023-02-27 12:54:50,434] Trial 0 finished with value: 99.1336518 and parameters: {'max_depth': 9, 'eta': 0.021650365985785874, 'colsample_bytree': 0.7701724233547466, 'subsample': 0.6479520691368729, 'min_child_weight': 9, 'n_estimators': 543, 'num_leaves': 65, 'lambda': 0.0034708878626066317, 'alpha': 0.00012843839558594053}. Best is trial 0 with value: 99.1336518.
[I 2023-02-27 12:55:17,176] Trial 1 finished with value: 100.4293898 and parameters: {'max_depth': 4, 'eta': 0.015455185967573677, 'colsample_bytree': 0.5097205689883119, 'subsample': 0.6363925501237795, 'min_child_weight': 4, 'n_estimators': 364, 'num_leaves': 14, 'lambda': 5.024282752261234e-08, 'alpha': 1.450363415489556e-08}. Best is trial 0 with value: 99.1336518.
[I 2023-02-27 12:55:51,345] Trial 2 finished with value: 99.7098022 and parameters: {'max_depth': 6, 'eta': 0.02051956938915479, 'colsample_bytree': 0.981456310883936, 'subsample': 0.9704754310737188, 'min_child_weight': 1, 'n_estimators': 188, 'num_leaves

[I 2023-02-27 12:59:25,754] A new study created in memory with name: no-name-ec945ab4-1a93-4b3b-bc37-65cc7eb5799f


[I 2023-02-27 12:59:25,742] Trial 9 finished with value: 96.802954 and parameters: {'max_depth': 8, 'eta': 0.015038157773592022, 'colsample_bytree': 0.5295818737052385, 'subsample': 0.886051206398718, 'min_child_weight': 2, 'n_estimators': 859, 'num_leaves': 38, 'lambda': 0.00053752096200205, 'alpha': 0.0009181782631851136}. Best is trial 9 with value: 96.802954.
Number of finished trials: 10
Best trial:
  Params: 
    max_depth: 8
    eta: 0.015038157773592022
    colsample_bytree: 0.5295818737052385
    subsample: 0.886051206398718
    min_child_weight: 2
    n_estimators: 859
    num_leaves: 38
    lambda: 0.00053752096200205
    alpha: 0.0009181782631851136


...

[I 2023-02-27 13:34:47,430] Trial 0 finished with value: 65.6155746 and parameters: {'max_depth': 8, 'eta': 0.029018982364431956, 'colsample_bytree': 0.6028981647984464, 'subsample': 0.985379292115323, 'min_child_weight': 7, 'n_estimators': 668, 'num_leaves': 13, 'lambda': 0.0002109199478079602, 'alpha': 7.507504190852514e-07}. Best is trial 0 with value: 65.6155746.
[I 2023-02-27 13:35:22,022] Trial 1 finished with value: 59.411956999999994 and parameters: {'max_depth': 7, 'eta': 0.025280150232470842, 'colsample_bytree': 0.43336814536132523, 'subsample': 0.9069433705616922, 'min_child_weight': 3, 'n_estimators': 263, 'num_leaves': 47, 'lambda': 0.8237586297696731, 'alpha': 0.006616402918689265}. Best is trial 1 with value: 59.411956999999994.
[I 2023-02-27 13:35:55,538] Trial 2 finished with value: 70.61387260000001 and parameters: {'max_depth': 7, 'eta': 0.024609463733090822, 'colsample_bytree': 0.6446691371804997, 'subsample': 0.9925716522247593, 'min_child_weight': 4, 'n_estimators

[I 2023-02-27 13:40:02,160] A new study created in memory with name: no-name-261d5e1f-733b-4097-902b-adfdd539be21


[I 2023-02-27 13:40:02,136] Trial 9 finished with value: 63.1281724 and parameters: {'max_depth': 5, 'eta': 0.012732263023973178, 'colsample_bytree': 0.8095807793923429, 'subsample': 0.8494569896670227, 'min_child_weight': 8, 'n_estimators': 635, 'num_leaves': 9, 'lambda': 0.12721404653631804, 'alpha': 8.398191888816665e-07}. Best is trial 1 with value: 59.411956999999994.
Number of finished trials: 10
Best trial:
  Params: 
    max_depth: 7
    eta: 0.025280150232470842
    colsample_bytree: 0.43336814536132523
    subsample: 0.9069433705616922
    min_child_weight: 3
    n_estimators: 263
    num_leaves: 47
    lambda: 0.8237586297696731
    alpha: 0.006616402918689265


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2023-02-27 13:40:36,521] Trial 0 finished with value: 49.586273199999994 and parameters: {'max_depth': 8, 'eta': 0.02165448738143488, 'colsample_bytree': 0.6911257503711734, 'subsample': 0.7338142730259565, 'min_child_weight': 5, 'n_estimators': 679, 'num_leaves': 45, 'lambda': 1.1970061929273557e-08, 'alpha': 1.1920992555598114e-07}. Best is trial 0 with value: 49.586273199999994.
[I 2023-02-27 13:41:11,221] Trial 1 finished with value: 47.748608600000004 and parameters: {'max_depth': 8, 'eta': 0.023102256961358585, 'colsample_bytree': 0.6815521743705131, 'subsample': 0.627534802650392, 'min_child_weight': 4, 'n_estimators': 761, 'num_leaves': 203, 'lambda': 0.131818273983481, 'alpha': 2.188439815587961e-05}. Best is trial 1 with value: 47.748608600000004.
[I 2023-02-27 13:41:41,288] Trial 2 finished with value: 49.256957199999995 and parameters: {'max_depth': 5, 'eta': 0.019508570017749857, 'colsample_bytree': 0.40796370684872707, 'subsample': 0.9624898965569825, 'min_child_weight

## Evaluate one model approach

In [210]:
from sklearn.metrics import mean_absolute_error

with open('/content/drive/MyDrive/params.json', 'r') as f:
    params = json.load(f)

dtrain = xgb.DMatrix(X_train, y_train.loc[:, ['max_usage']])
dtest  = xgb.DMatrix(X_test, y_test.loc[:, ['max_usage']])

gbm = xgb.train(params, dtrain, num_boost_round=1000)
prediction = gbm.predict(xgb.DMatrix(X_test)).astype(np.int64)
mean_absolute_error(prediction, y_test['max_usage'])


102.19111111111111

## Evaluate multi-model approach

In [209]:
with open('/content/drive/MyDrive/params_multi.json', 'r') as f:
    params = json.load(f)

predictions = []

for i, param in enumerate(params.values()):

    X_train_id = X_train.loc[X_train['municipality_id']==i]
    y_train_id = y_train.loc[y_train['municipality_id']==i, ['max_usage']]

    dtrain_id = xgb.DMatrix(X_train_id, y_train_id)

    X_test_id = X_test[X_test['municipality_id']==i]
    y_test_id = y_test.loc[y_test['municipality_id']==i, ['max_usage']]
    
    model = xgb.train(param, dtrain_id, num_boost_round=1000)
    prediction = model.predict(xgb.DMatrix(X_test_id))
    predictions.extend(prediction)

mean_absolute_error(predictions, y_test['max_usage'])

684.3571352767945